In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

## combining 2 page of table data together into 1 big table!

In [10]:
def get_content_value(row):
  if (row.find('li')):
    return [x.get_text(" ",strip=True).replace("\xa0"," ").replace("[1]","").strip() for x in row.find_all('li')]
  else:
    return row.find('td').get_text(" ",strip=True).replace("\xa0"," ").replace("[1]","").strip()
def clean_tags(soup):
  for tag in soup.find_all('sup'):
    tag.decompose()
def get_info_box(url):
  movie_info={}
  r = requests.get(url)
  soup = bs(r.content)
  info_box = soup.find('table',attrs={'class':'infobox vevent'})
  info_row = info_box.find_all('tr')
  clean_tags(soup)
  for index,row in enumerate(info_row):
      if (index==0):
        movie_info['title']=row.find('th').get_text(" ",strip=True)     #strip would remove the extra spaces and " " will be used as a join work
      elif (index==1):
        continue
      else:
        movie_info[row.find('th').get_text(" ",strip=True)]=get_content_value(row)
  return movie_info

In [ ]:
def get_row(r):
    return [x.get_text().replace('\n','') for x in r.find_all('td')]

r2 = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup2 = bs(r2.content)
movie_list_tables = soup2.find_all('table',attrs={'class':'wikitable sortable'})
column_name = [x.get_text().replace("\n","") for x in movie_list_tables[0].find_all('th')]
rows=[]
for table in movie_list_tables[0:9]:
    tr=table.find_all('tr')
    for row in tr:
      if (row.find('td')):
        try:
          page = 'https://en.wikipedia.org/'+row.select('td i a')[0].get('href')
          rows.append(get_info_box(page))
        except Exception as e:
          print(e)
      else:
        continue
rows

In [ ]:
df = pd.DataFrame(rows)
df.to_csv('all-disney-movies.csv',index=False)

In [ ]:
# creating json for easy load/save

import json
def save_data(title,data):
    with open(title,'w',encoding='utf-8') as f:
        json.dump(data,f,ensure_ascii=False,indent=2)
def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)
    
save_data("all-disney-movies.json",rows)

## Normal methods to extract info from a given movie

In [ ]:
r=requests.get('https://en.wikipedia.org/wiki/Toy_Story_3')
soup = bs(r.content)
info_box = soup.find('table',attrs={'class':'infobox vevent'})
info_row = info_box.find_all('tr')
for row in info_row:
    print(row.prettify())

In [ ]:
# this block helps to extract info from a movie wikipedia page

def get_content_value(row):
  if (row.find('li')):
    return [x.get_text(" ",strip=True).replace("\xa0"," ").replace("[1]","").strip() for x in row.find_all('li')]
  else:
    return row.find('td').get_text(" ",strip=True).replace("\xa0"," ").replace("[1]","").strip()
movie_info={}
for index,row in enumerate(info_row):
    if (index==0):
      movie_info['title']=row.find('th').get_text(" ",strip=True)     #strip would remove the extra spaces and " " will be used as a join work
    elif (index==1):
      continue
    else:
      movie_info[row.find('th').get_text(" ",strip=True)]=get_content_value(row)
movie_info

## Below block creates a dataframe with just 3 basic columns from the main wiki page

In [ ]:
def get_row(r):
    return [x.get_text().replace('\n','') for x in r.find_all('td')]

r2 = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup2 = bs(r2.content)
movie_list_tables = soup2.find_all('table',attrs={'class':'wikitable sortable'})
column_name = [x.get_text().replace("\n","") for x in movie_list_tables[0].find_all('th')]
rows=[]
for table in movie_list_tables:
    tr=table.find_all('tr')
    for row in tr:
      if (row.find('td')):
        if (len(row.find_all('td'))>3):
          continue
        else:
          rows.append(get_row(row))
      else:
        continue

df = pd.DataFrame(rows,columns=column_name)
df

## Basic Data Cleaning

In [91]:
df = pd.read_csv('all-disney-movies.csv')

#removing extra things from budget and box office

df['Budget'] = df['Budget'].fillna("")
df['Budget'] = [' '.join(x.split(' ')[:2]) for x in df['Budget']]
df['Budget'] = [x.split(' [')[0]  for x in df['Budget']]
df['Budget'] = df['Budget'].replace('[\$,]', '', regex=True)

df['Box office'] = df['Box office'].fillna("")
df['Box office'] = [x.split('[')[0].strip() for x in df['Box office']]
df['Box office'] = [x.split('(')[0].strip() for x in df['Box office']]
df['Box office'] = df['Box office'].replace('[\$,]', '', regex=True)

df['Directed by']=df['Directed by'].fillna("")

,Budget,Box office
0,1.5 million,418 million
1,2.6 million,164 million
2,2.28 million,76.4–83.3 million
3,600000,960000
4,950000,>1.3 million
...,...,...
507,295 million,384 million
508,150 million,117.5 million
509,,
510,175–200 million,244.1 million
